In [0]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/sukesh167/neural_nets/master/datsets/Churn_Modelling.csv')
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)
target=df.pop('Exited')
df['Gender'] = pd.Categorical(df['Gender'])
df['Gender'] = df.Gender.cat.codes
df['Geography'] = pd.Categorical(df['Geography'])
df['Geography'] = df.Geography.cat.codes
x_train,x_test,y_train,y_test=train_test_split(df,target,train_size=0.8)

In [0]:
x_train= torch.tensor(x_train.values)
y_train= torch.tensor(y_train.values)
y_train=y_train.view(y_train.size(0), -1)
x_test= torch.tensor(x_test.values)
y_test= torch.tensor(y_test.values)
y_test=y_test.view(y_test.size(0), -1)

In [0]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.ann=torch.nn.Sequential(
      torch.nn.Linear(10,10,bias=True),
      torch.nn.ReLU(inplace=True),
      torch.nn.Linear(10,10,bias=True),
      torch.nn.ReLU(inplace=True),
      torch.nn.Linear(10,1,bias=True),
      torch.nn.Sigmoid()
    )
  def forward(self,x):
    x=self.ann(x)
    return(x)   

In [0]:
ann=Net().cuda()
optimizer=torch.optim.SGD(ann.parameters(),lr=0.05)
loss=torch.nn.BCELoss().cuda()

In [0]:
def train(epoch,x_train,y_train,x_test,y_test):
  ann.train()
  tr_loss=0
  x_train=torch.autograd.Variable(x_train).cuda()
  y_train=torch.autograd.Variable(y_train).cuda()
  x_val=torch.autograd.Variable(x_test).cuda()
  y_val=torch.autograd.Variable(y_test).cuda()
  optimizer.zero_grad()
  out_train=ann(x_train.float())
  out_val=ann(x_val.float())
  loss_train=loss(out_train,y_train.float())
  loss_val=loss(out_val,y_val.float())
  loss_train.backward()
  optimizer.step()
  print('Epoch: ',epoch,'/t','acc: ',accuracy_score(y_train.detach().cpu(), np.argmax(list(out_train.detach().cpu().numpy()),axis=1)),'/t','loss: ',loss_train,'/t','val_acc: ',accuracy_score(y_val.detach().cpu(),np.argmax(list(out_val.detach().cpu().numpy()),axis=1)),'/t','val_loss: ',loss_val)

In [0]:
import numpy as np
for i in range(50):
  train(i,x_train,y_train,x_test,y_test)